# Miditok library anschauen und untersuchen, ob damit unser gewünschtes ziel von Tokenvorhersage für die Linke hand für einen bestimmten zeitabschnitt gemacht werden kann

In [1]:
import mido

# set the left_H tracks to instrument 1 to have the tokens delimited by program

def change_midi_program(midi_file_path, track_number, program_number, new_midi_file_path):
    """
    Change the program number of a specified track in a MIDI file.

    Parameters:
    - midi_file_path (str): Path to the input MIDI file.
    - track_number (int): The track number to change.
    - program_number (int): The new program number to set (0 for Acoustic Grand Piano, 1 for Bright Acoustic Piano, etc.).
    - new_midi_file_path (str): Path to save the modified MIDI file.
    """
    # Load the MIDI file
    midi = mido.MidiFile(midi_file_path)

    # Change the program number of the specified track
    for i, track in enumerate(midi.tracks):
        if i == track_number:
            new_messages = []
            for msg in track:
                if msg.type == 'program_change':
                    new_messages.append(mido.Message('program_change', program=program_number, time=msg.time))
                else:
                    new_messages.append(msg)
            midi.tracks[i] = new_messages

    # Save the modified MIDI file
    midi.save(new_midi_file_path)

    print(f"Track {track_number} program changed to {program_number} and saved as {new_midi_file_path}")

filepath = "/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/miditok_tokenization/tokenizer_test/test_files/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav-split.midi"

filepath_out = "/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/miditok_tokenization/tokenizer_test/test_files/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav-split_program_1.midi"

change_midi_program(filepath, 0, 1, filepath_out)


Track 0 program changed to 1 and saved as /home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/miditok_tokenization/tokenizer_test/test_files/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav-split_program_1.midi


In [2]:
# Create a tokenizer:
from miditok import Structured, REMI, TokenizerConfig, MMM
from pathlib import Path
from miditok import TokSequence

# Our parameters
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 8, (4, 12): 4},
    "num_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": True,
    "use_rests": False,
    "use_tempos": True,
    "use_time_signatures": True,
    "use_programs": True,
    "one_token_stream_for_programs": True,
    "num_tempos": 32,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)

# Creates the tokenizer
tokenizer = Structured(config)
# find midi files
midi_paths = list(Path("/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/miditok_tokenization/tokenizer_test/test_files").glob("**/*program_1.midi"))

# Builds the vocabulary with BPE
tokenizer.train(vocab_size=30000, files_paths=midi_paths)

/tmp/ipykernel_27738/267673723.py:21: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  config = TokenizerConfig(**TOKENIZER_PARAMS)


In [3]:
toksequence = tokenizer(midi_paths[0])

print(toksequence.tokens)

# save back to midi:
generated_midi = tokenizer(toksequence)
generated_midi.dump_midi(Path("/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/miditok_tokenization/tokenizer_test/test_files", "decoded.mid"))

['TimeShift_1.5.8', 'Program_0', 'Pitch_42', 'Velocity_71', 'Duration_1.1.8', 'TimeShift_1.0.8', 'Program_0', 'Pitch_49', 'Velocity_79', 'Duration_0.6.8', 'TimeShift_0.5.8', 'Program_0', 'Pitch_57', 'Velocity_83', 'Duration_0.2.8', 'TimeShift_0.3.8', 'Program_0', 'Pitch_57', 'Velocity_87', 'Duration_0.3.8', 'TimeShift_0.3.8', 'Program_0', 'Pitch_49', 'Velocity_83', 'Duration_0.3.8', 'TimeShift_0.3.8', 'Program_0', 'Pitch_42', 'Velocity_83', 'Duration_0.2.8', 'TimeShift_0.3.8', 'Program_0', 'Pitch_42', 'Velocity_75', 'Duration_0.5.8', 'TimeShift_0.3.8', 'Program_0', 'Pitch_49', 'Velocity_79', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Program_0', 'Pitch_57', 'Velocity_75', 'Duration_0.1.8', 'TimeShift_0.1.8', 'Program_0', 'Pitch_61', 'Velocity_95', 'Duration_0.3.8', 'TimeShift_0.2.8', 'Program_0', 'Pitch_42', 'Velocity_63', 'Duration_1.1.8', 'TimeShift_0.0.1', 'Program_0', 'Pitch_66', 'Velocity_87', 'Duration_3.4.8', 'TimeShift_0.7.8', 'Program_0', 'Pitch_49', 'Velocity_51', 'Duration_0.4.8'